##This is a notebook to demonstrate using bag of words BERT on base data

In [ ]:
!git clone https://github.com/adb262/transformers.git
%cd transformers
!pip install -r requirements.txt

In [2]:
import torch
import time

from bowBERT import BOWBERT
from functools import partial
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.functional import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#Import basic data, create iterators, vocab

In [3]:
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

tokenizer = get_tokenizer('basic_english')

In [4]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)
  
pad_token_idx = 2
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<cls>", "<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

In [5]:
text_pipeline = lambda x: vocab(tokenizer("<cls> " + x))
label_pipeline = lambda x: int(x) - 1

#Establish necessary variables, create model

In [6]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
LEARNING_RATE = 0.0003

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BOWBERT(vocab_size=vocab_size, n_encoders=2, d_model=64, n_heads=8, out_size=num_class, pad_token_idx=pad_token_idx).to(device)

In [7]:
def collate_batch(batch):
    """Collate function for dataloaders."""
    label_list, text_list = [], []
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=pad_token_idx)
    return label_list.to(device), text_list.to(device)

#Declare hyperparameters, dataloaders, and train

In [8]:
optimizer = partial(torch.optim.Adam, lr=LEARNING_RATE)(model.parameters())
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
criterion = nn.CrossEntropyLoss()
accuracy = []
log_interval = 10

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        one_hot_labels = nn.functional.one_hot(label, num_class).float()
        predicted_label = model(text)
        loss = criterion(predicted_label, one_hot_labels)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            accuracy.append(total_acc/total_count)
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
EPOCHS = 10
BATCH_SIZE = 128

total_accu = None

num_train = int(len(train_dataset) * 0.90)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      # scheduler.step()
      pass
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)